### Load module

In [2]:
import os
import sys
# sys.path.insert(0, os.path.abspath('../novelgraphs/'))

# import novelgraphs as ng

import pandas as pd
from collections import defaultdict, Counter
from itertools import combinations
import networkx as nx
from operator import itemgetter
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
text = pd.read_pickle('fight_text.pickle')

In [3]:
import json
with open('say.json') as file:
    say = json.load(file)

In [4]:
%%time
# corenlp = ng.annotators.CoreNLP()
# nernpid = ng.annotators.NerNpID()
quote = ng.annotators.Quote()
first_person = ng.annotators.FirstPerson()
character = ng.annotators.Character()
dialog = ng.annotators.Dialog()
pipeline = ng.annotators.Pipeline([quote, first_person, character, dialog])
pipeline.annotate(text)

CPU times: user 2min 3s, sys: 536 ms, total: 2min 4s
Wall time: 2min 5s


# Interactions

### Dinamic

In [1]:
def simple_interactions(table):
    character_list = []
    for i in table.index:
        if table.CharacterID.loc[i] is not None:
    #         print(table.Token.loc[i], table.SentenceID.loc[i])
            sent_numb = table.loc[i, 'SentenceID']
            character_list.append([table.CharacterID.loc[i], 
                                (table[table.SentenceID == sent_numb].index[0], 
                                  table[table.SentenceID == sent_numb].index[-1]), table.SentenceID.loc[i]])
    return character_list

In [44]:
def dinamic_interaction_in_sequence(table):
    list_of_pairs = []
    for i in table.index:
        if table.CharacterID.loc[i] is not None:
            if ((table.CharacterID.loc[i+2] is not None) & 
                ((table.Pos.loc[i+1] in ['VBD', 'VBN', 'VBP']) or (table.Lemma.loc[i+1] in say))):
                sent_numb = table.loc[i, 'SentenceID']
                list_of_pairs.append([(table.CharacterID.loc[i], table.CharacterID.loc[i+2]), 
                                    (table[table.SentenceID == sent_numb].index[0], 
                                      table[table.SentenceID == sent_numb].index[-1])])
    return list_of_pairs

In [46]:
(dinamic_interaction_in_sequence(text.tags))[0]

[(68, 10), (1238, 1241)]

In [73]:
def dinamic_interaction_dependences(table):
    list_of_pairs_dependences = []
    for s in table.SentenceID.unique():
        sentence = table.loc[table.SentenceID == s]
        for i in sentence.index:
            if (sentence.Pos.loc[i] in ['VBD', 'VBN', 'VBP'] or sentence.Lemma.loc[i] in say):
                left = sentence.loc[:i-1]
                left_el = left[~sentence.CharacterID.isnull()].index
                right = sentence.loc[i:]
                right_el = right[~sentence.CharacterID.isnull()].index
                for e in left_el:
                    if sentence.DepParse.loc[e] == sentence.TokenID.loc[i]:
                        for r in right_el:
                            if sentence.DepParse.loc[r] == sentence.TokenID.loc[i]:
                                list_of_pairs_dependences.append([(sentence.CharacterID.loc[e], 
                                                                    sentence.CharacterID.loc[r]), 
                                                                  (sentence.index[0], sentence.index[-1])])
    return list_of_pairs_dependences

In [78]:
# dinamic_interaction_dependences(text.tags)

### Dialog\context

In [60]:
def get_interactions_from_conversation(table):
    persons_in_dialog_list = []
    persons_in_dialog_set = []
    persons_in_dialog_combinations = []
    ids = [n for el in [[table[table.DialogID == i].index[0], 
            table[table.DialogID == i].index[-1]] for i in range(table.DialogID.max())] for n in el]
    dialog_ids = [[0,ids[0]]] + [ids[1:-1][i:i+2] for i in range(0, len(ids[1:-1]), 2)] + [[ids[-1], table.index.max()]]
    for el in dialog_ids:
        if len(table.CharacterID.loc[el[0]:el[1]][(~(table.CharacterID.isnull()))]) > 1:
            persons_in_dialog_list.append([(tuple(table.CharacterID.loc[el[0]:el[1]][(~(table.CharacterID.isnull()))])),
                              (el[0], el[1])])
            persons_in_dialog_set.append([(tuple(set(table.CharacterID.loc[el[0]:el[1]][(~(table.CharacterID.isnull()))]))),
                                      (el[0], el[1])])
            speakers = set(table.CharacterID.loc[el[0]:el[1]][(~(table.CharacterID.isnull()))])
            for pair in combinations(speakers, 2):
                persons_in_dialog_combinations.append([(tuple(pair)), (el[0], el[1])])
    return persons_in_dialog_list, persons_in_dialog_set, persons_in_dialog_combinations

In [61]:
dl, ds, dc = get_interactions_from_conversation(text.tags)

In [66]:
dl[:5]

[[(10, 10, 10, 68, 68, 10, 10), (0, 62)],
 [(68, 10, 68, 10, 10, 68, 10, 68, 10, 10), (204, 1102)],
 [(68,
   68,
   10,
   68,
   15,
   15,
   68,
   10,
   10,
   15,
   15,
   68,
   15,
   10,
   15,
   10,
   68,
   39,
   68,
   10,
   68,
   68,
   68,
   68,
   14,
   43,
   14,
   14,
   14,
   14),
  (1125, 1499)],
 [(14, 68, 14, 68), (1565, 1622)],
 [(68, 68, 68, 68, 68, 15, 15, 14, 14, 68, 14, 68, 14), (1645, 1983)]]

# Extractor

- Static

from Dinamic to Static by ***_flatten()***

- n предложений (любая длина окна)

# Aggregator